## Telecom churn case study: 

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn_data = pd.read_csv('https://cdn.upgrad.com/UpGrad/temp/0d723024-c76a-4242-ab5a-e4abaf7ea895/churn_data.csv')
customer_data = pd.read_csv('https://cdn.upgrad.com/UpGrad/temp/e53b4b48-976a-429a-a963-f352a6600955/customer_data.csv')
internet_data = pd.read_csv('https://cdn.upgrad.com/UpGrad/temp/518ef1d2-b74b-4325-a6de-30dd8c6cd959/internet_data.csv')


In [4]:
churn_data.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
customer_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [6]:
internet_data.head()

,customerID,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,No phone service,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,No,Fiber optic,No,No,No,No,No,No


In [235]:
telecom_data = pd.merge(customer_data, churn_data, on='customerID', how='inner')
telecom_data = pd.merge(telecom_data, internet_data, on='customerID', how='inner')
telecom_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,...,TotalCharges,Churn,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,Female,0,Yes,No,1,No,Month-to-month,Yes,Electronic check,...,29.85,No,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,Male,0,No,No,34,Yes,One year,No,Mailed check,...,1889.5,No,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,Male,0,No,No,2,Yes,Month-to-month,Yes,Mailed check,...,108.15,Yes,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,Male,0,No,No,45,No,One year,No,Bank transfer (automatic),...,1840.75,No,No phone service,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Female,0,No,No,2,Yes,Month-to-month,Yes,Electronic check,...,151.65,Yes,No,Fiber optic,No,No,No,No,No,No


In [236]:
telecom_data.shape

(7043, 21)

In [237]:
telecom_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
dtypes: float64(1), int64(2), obj

### Data cleaning: 

In [238]:
telecom_data.Churn.unique()

array(['No', 'Yes'], dtype=object)

In [239]:
telecom_data.Partner.unique()

array(['Yes', 'No'], dtype=object)

In [240]:
telecom_data.gender.unique()

array(['Female', 'Male'], dtype=object)

In [256]:
class CleanData:
    def __init__(self, _data):
        self.data = _data

    def clean(self):
        self.map_gender()
        for _key in [
                'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
                'Churn'
        ]:
            self.map_yes_or_no(_key)
        self.get_dummies()
        self.set_total_charge()
        return self.data

    def set_total_charge(self):
        self.data.TotalCharges = self.data.TotalCharges.convert_objects(convert_numeric=True)

    def map_gender(self):
        self.data['gender'] = self.data['gender'].map({'Female': 0, 'Male': 1})

    def get_dummies(self):
        _keys = [
            'Contract', 'PaymentMethod', 'DeviceProtection', 'StreamingMovies',
            'StreamingTV', 'TechSupport', 'OnlineBackup', 'OnlineSecurity',
            'InternetService', 'MultipleLines'
        ]
        dummies = pd.get_dummies(self.data[_keys], drop_first=True, prefix=_keys)
        self.data = pd.concat([self.data, dummies], axis=1)
        self.data.drop(columns=_keys, inplace=True, axis=1)

    def map_yes_or_no(self, _key):
        self.data[_key] = self.data[_key].map({'Yes': 1, 'No': 0})
    
    def drop_null_charges(self):
        self.data = self.data[~np.isnan(self.data.TotalCharges)]

In [248]:
telecom_data = pd.merge(customer_data, churn_data, on='customerID', how='inner')
telecom_data = pd.merge(telecom_data, internet_data, on='customerID', how='inner')
klass_obj = CleanData(telecom_data)
klass_obj.clean()
telecom_data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,...,TechSupport_No internet service,TechSupport_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,OnlineSecurity_No internet service,OnlineSecurity_Yes,InternetService_Fiber optic,InternetService_No,MultipleLines_No phone service,MultipleLines_Yes
0,7590-VHVEG,0,0,1,0,1,0,1,29.85,29.85,...,0,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,1,0,0,0,34,1,0,56.95,1889.50,...,0,0,0,0,0,1,0,0,0,0
2,3668-QPYBK,1,0,0,0,2,1,1,53.85,108.15,...,0,0,0,1,0,1,0,0,0,0
3,7795-CFOCW,1,0,0,0,45,0,0,42.30,1840.75,...,0,1,0,0,0,1,0,0,1,0
4,9237-HQITU,0,0,0,0,2,1,1,70.70,151.65,...,0,0,0,0,0,0,1,0,0,0


In [249]:
telecom_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 32 columns):
customerID                               7043 non-null object
gender                                   7043 non-null int64
SeniorCitizen                            7043 non-null int64
Partner                                  7043 non-null int64
Dependents                               7043 non-null int64
tenure                                   7043 non-null int64
PhoneService                             7043 non-null int64
PaperlessBilling                         7043 non-null int64
MonthlyCharges                           7043 non-null float64
TotalCharges                             7032 non-null float64
Churn                                    7043 non-null int64
Contract_One year                        7043 non-null uint8
Contract_Two year                        7043 non-null uint8
PaymentMethod_Credit card (automatic)    7043 non-null uint8
PaymentMethod_Electronic check    

#### checking for outliers: 

In [253]:
telecom_data[['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen']].describe()

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen
count,7043.000000,7043.000000,7032.000000,7043.000000
mean,32.371149,64.761692,2283.300441,0.162147
std,24.559481,30.090047,2266.771362,0.368612
min,0.000000,18.250000,18.800000,0.000000
25%,9.000000,35.500000,401.450000,0.000000
50%,29.000000,70.350000,1397.475000,0.000000
75%,55.000000,89.850000,3794.737500,0.000000
max,72.000000,118.750000,8684.800000,1.000000


In [255]:
round(100*(telecom_data.isnull().sum()/len(telecom_data.index)), 2)

customerID                               0.00
gender                                   0.00
SeniorCitizen                            0.00
Partner                                  0.00
Dependents                               0.00
tenure                                   0.00
PhoneService                             0.00
PaperlessBilling                         0.00
MonthlyCharges                           0.00
TotalCharges                             0.16
Churn                                    0.00
Contract_One year                        0.00
Contract_Two year                        0.00
PaymentMethod_Credit card (automatic)    0.00
PaymentMethod_Electronic check           0.00
PaymentMethod_Mailed check               0.00
DeviceProtection_No internet service     0.00
DeviceProtection_Yes                     0.00
StreamingMovies_No internet service      0.00
StreamingMovies_Yes                      0.00
StreamingTV_No internet service          0.00
StreamingTV_Yes                   